In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing library
import pandas as pd
import numpy as np

In [ ]:
import os

# # Paths
# original_dataset_path = "/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train"

# file_count = sum(len(files) for _, _, files in os.walk(original_dataset_path))

# print(f"Total number of training files: {file_count}")


In [ ]:
import os
import shutil
import random

# Paths to given ooriginal dataset and the one we will derive the only we need from dataset
original_dataset_path = "/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train"
new_dataset_path = "/kaggle/working/Choosen_ILSVRC"

# Checking new dataset directories exist
os.makedirs(f"{new_dataset_path}/train", exist_ok=True)
os.makedirs(f"{new_dataset_path}/test", exist_ok=True)
os.makedirs(f"{new_dataset_path}/val", exist_ok=True)

# Selecting 100 classes
all_classes = sorted(os.listdir(original_dataset_path))

# Randomly choosing 100 classes
selected_classes = random.sample(all_classes, 100)

# Step 2: Collecting  images
images_per_class = 500

total_images = images_per_class * len(selected_classes) #100*500==500000 total images
all_images = [] #list to store image paths.

for class_name in selected_classes:
    class_path = os.path.join(original_dataset_path, class_name)
    images = sorted(os.listdir(class_path))[:images_per_class]  # Check 500 per class
    all_images.extend([(class_name, img) for img in images])

# To ensure random distribution of all images we collected
random.shuffle(all_images)

# Step 3: Split dataset
train_split = 30000
test_split = 10000
val_split = 10000

train_images = all_images[:train_split]
test_images = all_images[train_split:train_split + test_split]
val_images = all_images[train_split + test_split:]




In [ ]:
# Function to copy images to respective folders
def copy_images(image_list, folder_name):
    for class_name, img in image_list:
        src_path = os.path.join(original_dataset_path, class_name, img)
        dest_dir = os.path.join(new_dataset_path, folder_name, class_name)
        os.makedirs(dest_dir, exist_ok=True)
        shutil.copy(src_path, dest_dir)


In [ ]:
# Step 4: Save to new directories
copy_images(train_images, "train")
print("train folder is successfully created and saved.")


train folder is successfully created and saved.


In [ ]:
copy_images(test_images, "test")
print("test is successfully created and saved.")


test is successfully created and saved.


In [ ]:
copy_images(val_images, "val")
print("val is successfully created and saved.")


val is successfully created and saved.


In [ ]:

print("Finally, Dataset is successfully created and saved.")

Finally, Dataset is successfully created and saved.


In [ ]:

# Function to count images in a directory
def count_images(folder_path):
    total_images = 0
    for class_folder in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_folder)
        if os.path.isdir(class_path):
            total_images += len(os.listdir(class_path))
    return total_images

# Count images in each dataset split
train_count = count_images(os.path.join(new_dataset_path, "train"))
test_count = count_images(os.path.join(new_dataset_path, "test"))
val_count = count_images(os.path.join(new_dataset_path, "val"))

# Print results
print(f"Training images: {train_count}")
print(f"Testing images: {test_count}")
print(f"Validation images: {val_count}")
print(f"Total images: {train_count + test_count + val_count}")


Training images: 30000
Testing images: 10000
Validation images: 10000
Total images: 50000


In [ ]:
# Check number of classes
def count_classes(folder_path):
    return len([d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))])

# Check number of images in each class
def count_images_per_class(folder_path):
    class_counts = {}
    for class_folder in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_folder)
        if os.path.isdir(class_path):
            class_counts[class_folder] = len(os.listdir(class_path))
    return class_counts


In [ ]:
#COUNT CLASSES IN EACH FOLDER WE SEPARATED
train_classes = count_classes(os.path.join(new_dataset_path, "train"))
test_classes = count_classes(os.path.join(new_dataset_path, "test"))
val_classes = count_classes(os.path.join(new_dataset_path, "val"))

print(f"Total classes in train: {train_classes}")
print(f"Total classes in test: {test_classes}")
print(f"Total classes in val: {val_classes}")



Total classes in train: 100
Total classes in test: 100
Total classes in val: 100


In [ ]:
# Count total images in each split
train_images = count_images_per_class(os.path.join(new_dataset_path, "train"))
test_images = count_images_per_class(os.path.join(new_dataset_path, "test"))
val_images = count_images_per_class(os.path.join(new_dataset_path, "val"))

print(f"Total images in train: {sum(train_images.values())}")
print(f"Total images in test: {sum(test_images.values())}")
print(f"Total images in val: {sum(val_images.values())}")



Total images in train: 30000
Total images in test: 10000
Total images in val: 10000


In [ ]:
import tensorflow as tf

# Define dataset paths
dataset_path = "/kaggle/working/Choosen_ILSVRC"

# ImageNet standard input size for AlexNet
IMG_SIZE = 224  # AlexNet expects 224x224 input size
BATCH_SIZE = 128

# ImageNet mean and standard deviation for normalization
IMAGENET_MEAN = tf.constant([0.485, 0.456, 0.406])
IMAGENET_STD = tf.constant([0.229, 0.224, 0.225])

def preprocess_image(image, label, training=True):
    if tf.shape(image)[-1] == 1:
        image = tf.image.grayscale_to_rgb(image)
    elif tf.shape(image)[-1] == 4:
        image = image[:, :, :3]

    # Resize image to 256x256
    image = tf.image.resize_with_pad(image, 256, 256)

    if training:
        image = tf.image.random_crop(image, (224, 224, 3))
        image = tf.image.random_flip_left_right(image)
    else:
        image = tf.image.central_crop(image, central_fraction=0.875)

    # Normalize
    image = tf.cast(image, tf.float32) / 255.0
    image = (image - IMAGENET_MEAN) / IMAGENET_STD

    return image, label


# Load datasets using TensorFlow's image_dataset_from_directory
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path + "/train",
    label_mode="int",
    image_size=(256, 256),
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path + "/val",
    label_mode="int",
    image_size=(256, 256),
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path + "/test",
    label_mode="int",
    image_size=(256, 256),
    batch_size=BATCH_SIZE,
    shuffle=False
)

train_ds = train_ds.map(lambda x, y: preprocess_image(x, y, training=True), num_parallel_calls=tf.data.AUTOTUNE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.map(lambda x, y: preprocess_image(x, y, training=False), num_parallel_calls=tf.data.AUTOTUNE).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.map(lambda x, y: preprocess_image(x, y, training=False), num_parallel_calls=tf.data.AUTOTUNE).prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:
# Check dataset structure
print(f"Training set: {len(train_ds)} batches")
print(f"Validation set: {len(val_ds)} batches")
print(f"Testing set: {len(test_ds)} batches")

Training set: 235 batches
Validation set: 79 batches
Testing set: 79 batches
